# Missing values

In [1]:
import polars as pl
import polars.selectors as cs

### Missing values in Polars
Missing values in Polars are represented with a `null` value for all dtypes.

In [2]:
df = pl.DataFrame(
    {
        'col1':[None,2,3,4],
        "col2":[None,None,5,6],
        "col3":[None,None,None,7]
    }
)
df

col1,col2,col3
i64,i64,i64
null,null,null
2,null,null
3,5,null
4,6,7


### Metadata on `null` values
Polars stores metadata about `null` values for each column in a `DataFrame`.

#### Null count
Polars stores a count of how many `null` values there are.

In [3]:
df.null_count()

col1,col2,col3
u32,u32,u32
1,2,3


### Finding `null` values

We use the `is_null` expression to find out whether each value is `null` and `is_not_null`.

In [4]:
df.select(
    pl.col("col1"),
    pl.col("col1").is_null().alias("is_null"),
    pl.col("col1").is_not_null().alias("is_not_null")
)

col1,is_null,is_not_null
i64,bool,bool
null,true,false
2,false,true
3,false,true
4,false,true


### Filtering by `null` values

#### Filtering on a single column

In [5]:
df.filter(
    pl.col("col1").is_not_null()
)

col1,col2,col3
i64,i64,i64
2,null,null
3,5,null
4,6,7


#### Filtering by `null` values in multiple columns

In [7]:
df.filter(
    pl.any_horizontal(pl.all().is_not_null())
)

col1,col2,col3
i64,i64,i64
2,null,null
3,5,null
4,6,7


In [8]:
df.filter(
    pl.all_horizontal(pl.all().is_not_null())
)

col1,col2,col3
i64,i64,i64
4,6,7


### Using the `drop_nulls` method

In [9]:
df.drop_nulls()

col1,col2,col3
i64,i64,i64
4,6,7


Drop nulls according to columns.

In [10]:
df.drop_nulls(subset=["col1", "col2"])

col1,col2,col3
i64,i64,i64
3,5,null
4,6,7


## Exercises

### Exercise 1
Count the number of `null` values in each row of the Titanic data

In [11]:
csv_file = "data/titanic.csv"
(
    pl.read_csv(csv_file)
    .null_count()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,177,0,0,0,0,687,2


Filter out the rows that are `null` from the `Cabin` column and count the null values for all columns again

In [12]:
pl.read_csv(csv_file).filter(
    pl.col("Cabin").is_not_null()
).null_count()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,19,0,0,0,0,0,2


### Exercise 2
Find all the rows for which the `Age` is `null`

In [14]:
pl.read_csv(csv_file).filter(
    pl.col("Age").is_null()
).head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
6,0,3,"""Moran, Mr. James""","""male""",null,0,0,"""330877""",8.4583,null,"""Q"""
18,1,2,"""Williams, Mr. Charles Eugene""","""male""",null,0,0,"""244373""",13.0,null,"""S"""
20,1,3,"""Masselmani, Mrs. Fatima""","""female""",null,0,0,"""2649""",7.225,null,"""C"""
27,0,3,"""Emir, Mr. Farred Chehab""","""male""",null,0,0,"""2631""",7.225,null,"""C"""
29,1,3,"""O'Dwyer, Miss. Ellen ""Nellie""""","""female""",null,0,0,"""330959""",7.8792,null,"""Q"""


Find all the rows for which neither the `Age` nor the `Cabin` is `null`.

Use the Selectors API (imported above as `cs`) to select the columns

In [15]:
import polars.selectors as cs

In [16]:
pl.read_csv(csv_file).filter(
    pl.any_horizontal(cs.matches("Age|Cabin").is_null())
).select(
    cs.matches("Age|Cabin")
)

Age,Cabin
f64,str
22.0,null
26.0,null
35.0,null
null,null
2.0,null
…,…
25.0,null
39.0,null
27.0,null
